In [1]:
#Find a way to save the tweet to disk

import json
import oauth2 as oauth
import os
import emoji as emo
import urllib.parse as parse
import csv
from sklearn.feature_extraction.text import CountVectorizer
from stemming.porter2 import stem
from string import punctuation
import numpy
import pymongo
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [2]:
#This function will use an online ready-made dictionary to find the sentiment of an emoji

#load csv info into dictionary
emoji_scores = {}
with open('Emoji_Sentiment_Data_v1.0.csv', newline = '') as emoji_csv:
    emoji_reader = csv.reader(emoji_csv)
    next(emoji_reader, None)
    for row in emoji_reader:
        emoji = row[0]
        neg_score = float(int(row[4])/int(row[2]))
        neut_score = float(int(row[5])/int(row[2]))
        pos_score = float(int(row[6])/int(row[2]))
        sent_score = (-1 * neg_score + pos_score) - .305
        emoji_scores[emoji] = sent_score
        
#print(emoji_scores)
        

In [3]:
consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')

access_token = os.environ.get('ACCESS_TOKEN')
access_token_secret = os.environ.get('ACCESS_TOKEN_SECRET')

consumer = oauth.Consumer(key=consumer_key, secret=consumer_secret)
access_token = oauth.Token(key=access_token, secret=access_token_secret)
client = oauth.Client(consumer, access_token)



emo_tweets = {}
for i in range(0, 10):
    params = {'q' : 'Justin Bieber', 'count' : 100, 'lang' : 'en'}
    # See Twitter Search API documenation for description of cursoring for below if block
    if i > 0:
        params['max_id'] = max_id 
    timeline_endpoint_baseuri = 'https://api.twitter.com/1.1/search/tweets.json?'
    timeline_endpoint = timeline_endpoint_baseuri + parse.urlencode(params)
    print(timeline_endpoint)
    response, data = client.request(timeline_endpoint)
    tweets_results = json.loads(data)
    tweets_list = tweets_results['statuses']
    max_id = tweets_list[len(tweets_list) - 1]['id'] - 1
    for tweet in tweets_list:
        tweet_emojis = []
        for char in tweet['text']:
            if char in emo.UNICODE_EMOJI:
                tweet_emojis.append(char)
        if tweet_emojis:
            for emoji in tweet_emojis:
                tweet_emoji_scores = []
                try:
                    tweet_emoji_scores.append(emoji_scores[emoji])
                except KeyError as e:
                    print('No emoji %s' % e)
                    continue
            if tweet_emoji_scores:
                if max(tweet_emoji_scores) == max(numpy.absolute(tweet_emoji_scores)):
                    tweet_score = max(tweet_emoji_scores)
                else:
                    tweet_score = min(tweet_emoji_scores)
                emo_tweets[tweet['id']] = [tweet['text'], tweet_score]
    print('Emoji tweets processed: ' + str(len(emo_tweets.keys())))
            

for tweet in emo_tweets.values():
    print(tweet)
    print()

https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en
Emoji tweets processed: 0
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en&max_id=896134177900253183
No emoji '🏻'
No emoji '🤤'
Emoji tweets processed: 1
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en&max_id=896133950464073728
No emoji '🏻'
No emoji '🤤'
No emoji '🏻'
No emoji '🤤'
Emoji tweets processed: 6
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en&max_id=896133699384655872
No emoji '🤢'
No emoji '🤢'
No emoji '🤢'
No emoji '🤢'
No emoji '🤢'
No emoji '🏻'
Emoji tweets processed: 9
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en&max_id=896133467003420671
Emoji tweets processed: 11
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=100&lang=en&max_id=896133213986332671
Emoji tweets processed: 18
https://api.twitter.com/1.1/search/tweets.json?q=Justin+Bieber&count=1

In [3]:
# text preprocessing (partially from before partially from August)
mongo_uri = 'mongodb://heroku_xgnhblcr:' + os.environ.get('MONGODB_PASSWORD') + '@ds149511.mlab.com:49511/heroku_xgnhblcr'
client = pymongo.MongoClient(mongo_uri)
db = client.get_default_database()
myresults = list(db.Justin_Bieber.find())
tweets = []
emoji_tweets = []
for entry in myresults:
    tweets.append(entry['text'])
#Doing the next part based on the earlier REST-API-using cell
for tweet in tweets:
    tweet_emojis = []
    for char in tweet:
        if char in emo.UNICODE_EMOJI:
            tweet_emojis.append(char)
    if tweet_emojis:
        for emoji in tweet_emojis:
            tweet_emoji_scores = []
            try:
                tweet_emoji_scores.append(emoji_scores[emoji])
            except KeyError as e:
                print('No emoji score for %s' % e)
                continue
            if tweet_emoji_scores:
                if max(tweet_emoji_scores) == max(numpy.absolute(tweet_emoji_scores)):
                    tweet_score = max(tweet_emoji_scores)
                else:
                    tweet_score = min(tweet_emoji_scores)
                emoji_tweets.append([tweet, tweet_score])
    
print(emoji_tweets)

No emoji score for '🏼'
No emoji score for '🤔'
No emoji score for '🤤'
No emoji score for '🤣'
No emoji score for '🤣'
No emoji score for '🤣'
No emoji score for '🏾'
No emoji score for '🏼'
No emoji score for '🙄'
No emoji score for '🙄'
No emoji score for '🤷'
No emoji score for '🏾'
No emoji score for '🏼'
No emoji score for '🏽'
No emoji score for '🏽'
No emoji score for '🏵'
No emoji score for '🏵'
No emoji score for '🦄'
No emoji score for '🦄'
No emoji score for '🏼'
No emoji score for '🏼'
No emoji score for '🏼'
No emoji score for '🦄'
No emoji score for '🏵'
No emoji score for '🦄'
No emoji score for '🤔'
No emoji score for '🤔'
No emoji score for '🌪'
No emoji score for '🌪'
No emoji score for '🏵'
No emoji score for '🦄'
No emoji score for '🤴'
No emoji score for '🏻'
No emoji score for '🏼'
No emoji score for '🤸'
No emoji score for '🙃'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji score for '🏻'
No emoji sc

In [4]:
tweet_text= []
for tweet in emoji_tweets:
    tweet_text.append(tweet[0])
    
    
def preprocessor(tweet):
    
    if tweet.startswith('RT'):
        return ''
    #remove accents
    normalized = unicodedata.normalize('NFKD', tweet)
    if normalized != tweet:
        tweet = ''.join([c for c in normalized if not unicodedata.combining(c)])
        
    tweet = tweet.lower()
    
    #remove URLs
    tweet = re.sub(r'https?:\/\/.*?(\s|$)', '', tweet)
    tweet = re.sub(r'@\w*?(\s|$)', '', tweet)
    tweet = re.sub(r'#\w*?(\s|$)', '', tweet)
    
    #remove RT (retweet) - this doesn't seem to work since 'rt' is in the preprocessed data
    # This is unnecessary if I remove them in the beginning of the preprocessor
    tweet = re.sub('^rt\s', '', tweet)
    
    #Note - the preprocessor already seems to be removing emojis - no need for extra code
    
    # Regularize apostrophes from different parts of unicode
    tweet = re.sub('|'.join(['‘', '’', 'ʻ', 'ʼ']), '\'', tweet)
    
    return tweet
    


In [5]:
def tokenizer(tweet): #Currently troubleshooting function
    
    tokens = []
    token_pattern = '(^|(?!\w).)((\w|\')+)\W'
    token_matches = re.finditer(token_pattern, tweet)
    for token_match in token_matches:
        tokens.append(token_match.group(2))
    return tokens

print(tokenizer("Hard pass...I'll give as much respect as y'all did Barack...none 😒 https://t.co/UJcr4mrZRA"))

['Hard', "I'll", 'as', 'respect', "y'all", 'Barack', 'none', 'https', 't']


In [6]:
vectorizer = CountVectorizer(preprocessor=preprocessor, tokenizer=tokenizer) #Make binary?  Does this prevent it from picking up one word twice in a tweet?
fit_tr = vectorizer.fit_transform(tweet_text)
print(vectorizer.get_feature_names())
print(fit_tr)
#   for word in emoji_tweets.split(): #This and following line should be removed and use regex instead.  Follow Downloads/pak-paroubek.pdf for preprocessing ideas, as well as downloads that were downloaded at a similar time
#      if word[0:4] == 'http':
            
#CountVectorizer().build_tokenizer()(tweet_text) #stop_words param to 'english' once you have only Eng tweets, this was indented within for loop at the beginning of August

["'i'm", '100', '15a20', '2', '2009', '2010', '2012', '2013', '24', '250', '2u', '3', '4ever', '5', '61', '8', 'a', 'able', 'about', 'act', 'acting', 'actually', 'after', 'ahhh', 'alive', 'all', 'am', 'amazing', 'american', 'amor', 'amp', 'an', 'and', 'angel', 'angels', 'angrily', 'anllela', 'are', 'ariana', 'around', 'as', 'ass', 'at', 'attacked', 'attention', 'aurruz', 'authentic', 'awesome', 'babe', 'baby', 'be', 'beautiful', 'beauty', 'because', 'bedtime', 'been', 'before', 'being', 'beliber', 'beliebers', 'believe', 'believers', 'best', 'better', 'bich', 'bieber', 'biebs', 'big', 'biggest', 'bitch', 'black', 'boa', 'book', 'booty', 'boy', 'boyfriend', 'brotha', 'bruh', 'bts', 'but', 'by', 'byliber', 'can', 'cc', 'celeste', 'chance', 'change', 'check', 'cold', 'colombia', 'color', 'come', 'comes', 'comments', 'concert', 'condition', 'confident', 'could', 'cover', 'cream', 'cz', 'daddy', 'dances', 'darling', 'dat', 'david', 'davy', 'de', 'dead', 'deal', 'deals', 'deejay', 'defeated'

In [7]:
transformer = TfidfTransformer()  # This block is not correct
tfidf = transformer.fit_transform(fit_tr)

In [8]:
    tweet_emos = []
    for tweet in emoji_tweets:
        tweet_emos.append(tweet[1] > 0)
        
    
    #Cross-validation
    from sklearn.model_selection import cross_val_score
    print(type(tfidf.toarray()))
          
    scores = cross_val_score(MultinomialNB(alpha=.01), tfidf, tweet_emos, cv=5) #this takes forever at cv=10
    print(scores)  
    
    

<class 'numpy.ndarray'>
[ 0.64677223  0.54323995  0.58241758  0.5995116   0.62148962]


In [15]:
    
    
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    MultinomialNB(alpha=.01).fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

________________________________________________________________________________
Training: 


NameError: name 'clf' is not defined